In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
## Fundamental Data Analysis
funda_data = pd.read_csv('Fundamental_data.csv')
funda_data.columns

Index(['gvkey', 'permno', 'adate', 'qdate', 'public_date', 'pcf', 'dpr', 'npm',
       'gpm', 'roa', 'roe', 'capital_ratio', 'de_ratio', 'cash_ratio',
       'curr_ratio', 'inv_turn', 'pay_turn', 'sale_nwc', 'rd_sale',
       'staff_sale', 'accrual', 'PEG_trailing', 'TICKER'],
      dtype='object')

In [3]:
funda_data = funda_data.drop(columns=['public_date',"pcf",'staff_sale','PEG_trailing','gvkey','permno','adate'])
funda_data = funda_data.drop_duplicates()

In [35]:
def merge_left(funda_stock,name,freq):
    price = pd.read_csv('../stock_price/'+freq+'/'+name+'.csv')
    price['Date'] = price.Date.apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d'))
    close = price.loc[:,['Date']]
    df = pd.merge(funda_stock,close,how='right',left_on=['qdate'],right_on=['Date'])
    df.fillna(method='ffill',inplace=True)
    df.set_index('Date',inplace=True)
    df.dropna(inplace=True)
    df.drop(columns=['qdate','TICKER'],inplace=True)
    df.to_csv(freq+'/'+name+'.csv')

In there, we delete the unuseful columns named pulic_date, pcf, staff_sale, peg_trailing, gvkey, permno and drop the duplicates for each rows. We also delete the adate data out of the purpose to delete the unuseful information here. Finally, we gonna use the qdate to be the corresponding time to mark the ticker.

In [38]:
for name in pd.unique(funda_data.TICKER).tolist():
    funda_stock = funda_data.loc[funda_data.TICKER==name,:]
    funda_stock['qdate'] = funda_data.qdate.apply(lambda x:dt.datetime.strptime(x,'%m/%d/%Y'))
    if name == 'FB':
        name = 'META'
    elif name == 'BRK':
        name = 'BRK-B'
    elif name == 'GOOGL':
        name = 'GOOG'
    merge_left(funda_stock,name,'Daily')
    merge_left(funda_stock,name,'Weekly')
    merge_left(funda_stock,name,'Monthly')



/var/folders/ck/fdgds2ns7w3bfppbyfdv7x1m0000gn/T/ipykernel_59105/1594930208.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  funda_stock['qdate'] = funda_data.qdate.apply(lambda x:dt.datetime.strptime(x,'%m/%d/%Y'))
/var/folders/ck/fdgds2ns7w3bfppbyfdv7x1m0000gn/T/ipykernel_59105/1594930208.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  funda_stock['qdate'] = funda_data.qdate.apply(lambda x:dt.datetime.strptime(x,'%m/%d/%Y'))
/var/folders/ck/fdgds2ns7w3bfppbyfdv7x1m0000gn/T/ipykernel_59105/159493020

### Indicators Mergation

In [81]:
#Can be used to make up time among two interval, but unuseful now


# def make_up_time(df,freq):
#     len = df.shape[0]
#     print(len,df.shape)
#     df.qdate = df.qdate.apply(lambda x: dt.datetime.strptime(x,'%m/%d/%Y'))
#     original_time = df.qdate
#     for i in range(1,len):
#         cur = original_time.iloc[i]-dt.timedelta(days=1)
#         last = original_time.iloc[i-1]+dt.timedelta(days=1)
#         small_df = pd.DataFrame(columns=df.columns)
#         if freq.lower() == 'daily':
#             times_range = pd.date_range(last,cur,freq='D')
            
#         elif freq.lower() == 'monthly':
#             times_range = pd.date_range(last,cur,freq='M')
#         else:
#             times_range = pd.date_range(last,cur,freq='W')

#         small_df['qdate'] = times_range
#         df = pd.concat([df,small_df])
#         df.sort_values('qdate',inplace=True)
#         df.fillna(method='backfill',inplace=True)
#     return df